## Libraries, Dependencies and Parameters


_Install Required Libraries_


In [1]:
%pip install evaluate
%pip install openprompt
%pip install adapter-transformers==3.1.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 4.5 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.4/146.4 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.5 MB/s eta 0:00:00
  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.98
    Uninstalling sentencepiece-0.1.98:
      Successfully uninstalled sentencepiece-0.1.98
Note: you may need to restart the kernel to use updated packages.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 60.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 84.4 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
Note: you may need to restart the kernel to use updated packages.


_Dependencies_


In [2]:
import torch
import warnings
import pandas as pd
from prettytable import PrettyTable
from collections import defaultdict
from datasets import load_metric
from openprompt import PromptDataLoader, PromptForClassification
from openprompt.data_utils import InputExample
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.plms.utils import TokenizerWrapper
from transformers import XLMRobertaConfig, XLMRobertaTokenizer, XLMRobertaModel, XLMRobertaForMaskedLM, set_seed
from sklearn.model_selection import train_test_split

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
warnings.filterwarnings('ignore')

_Parameters_


In [4]:
model_type = "XLM-R"
technique = "Sentiment" #@param ["Sentiment", "Humor", "Hate-Speech"]
use_cuda = True
no_of_labels = 4 #Sentiment:4, Humor:2, Hate-Speech:3
validation_size = (1/9)
test_size = 0.1
split_random_state = 42
max_seq_length = 128
batch_size = 32
training_seed = 42 #@param [8, 42, 77]
script = "Char-Script-1.0"

## Dataset Preprocessing


In [5]:
df = pd.read_csv("/kaggle/input/sinhala-english-cmcs-dataset/annotated-script(all).csv")
df = df[['Sentence', technique, script]]
df.columns = ['Sentence', 'Label', script]

*Label Encoding*

In [6]:
df['Label'], uniq = pd.factorize(df['Label'])

*Train, Validation & Test Split*

In [7]:
X, y = df[['Sentence', script]], df[['Label']]
stratifying_col = y["Label"]
X_rem, X_test, y_rem, y_test = train_test_split(X, y, test_size=test_size, stratify=stratifying_col, random_state=split_random_state)
stratifying_col = y_rem["Label"]
X_train, X_validation, y_train, y_validation = train_test_split(X_rem, y_rem, test_size=validation_size, stratify=stratifying_col, random_state=split_random_state)

In [8]:
print("Train : Rows =", X_train.shape[0], ", Columns = ", X_train.shape[1])
print("y_train : Rows =", y_train.shape[0], ", Columns = ", y_train.shape[1])
print("X_validation : Rows =", X_validation.shape[0], ", Columns = ", X_validation.shape[1])
print("y_validation : Rows =", y_validation.shape[0], ", Columns = ", y_validation.shape[1])
print("X_test : Rows =", X_test.shape[0], ", Columns = ", X_test.shape[1])
print("y_test : Rows =", y_test.shape[0], ", Columns = ", y_test.shape[1])

Train : Rows = 10814 , Columns =  2
y_train : Rows = 10814 , Columns =  1
X_validation : Rows = 1352 , Columns =  2
y_validation : Rows = 1352 , Columns =  1
X_test : Rows = 1352 , Columns =  2
y_test : Rows = 1352 , Columns =  1


In [9]:
print("Labels :", ['Negative', 'Neutral', 'Positive', 'Conflict'])
print("Train :", y_train.groupby('Label').size().tolist())
print("Validation :", y_validation.groupby('Label').size().tolist())
print("Test :", y_test.groupby('Label').size().tolist())

Labels : ['Negative', 'Neutral', 'Positive', 'Conflict']
Train : [2913, 6877, 924, 100]
Validation : [364, 860, 116, 12]
Test : [364, 860, 116, 12]


## OpenPrompt


In [10]:
set_seed(training_seed)
torch.backends.cudnn.deterministic = True 

In [11]:
class MLMTokenizerWrapper(TokenizerWrapper):
    add_input_keys = ['input_ids', 'attention_mask', 'token_type_ids']

    @property
    def mask_token(self):
        return self.tokenizer.mask_token

    @property
    def mask_token_ids(self):
        return self.tokenizer.mask_token_id

    @property
    def num_special_tokens_to_add(self):
        if not hasattr(self, '_num_specials'):
            self._num_specials = self.tokenizer.num_special_tokens_to_add()
        return self._num_specials

    def tokenize_one_example(self, wrapped_example, teacher_forcing):
        wrapped_example, others = wrapped_example

        # for some dataset like SuperGLUE.COPA, the answer requires prediction an span of
        # the input. Or in generation tasks, we need to generate a piece of target_text.
        # In these case, it tokenized to the encoded_tgt_text for future use.
        encoded_tgt_text = []
        if 'tgt_text' in others:
            tgt_text = others['tgt_text']
            if isinstance(tgt_text, str):
                tgt_text = [tgt_text]
            for t in tgt_text:
                encoded_tgt_text.append(self.tokenizer.encode(t, add_special_tokens=False))

        mask_id = 0 # the i-th the mask token in the template.

        encoder_inputs = defaultdict(list)
        for piece in wrapped_example:
            if piece['loss_ids']==1:
                if teacher_forcing: # fill the mask with the tgt task
                    raise RuntimeError("Masked Language Model can't perform teacher forcing training!")
                else:
                    encode_text = [self.mask_token_ids]
                mask_id += 1

            if piece['text'] in self.special_tokens_maps.keys():
                to_replace = self.special_tokens_maps[piece['text']]
                if to_replace is not None:
                    piece['text'] = to_replace
                else:
                    raise KeyError("This tokenizer doesn't specify {} token.".format(piece['text']))

            if 'soft_token_ids' in piece and piece['soft_token_ids']!=0:
                encode_text = [0] # can be replace by any token, since these token will use their own embeddings
            else:
                encode_text = self.tokenizer.encode(piece['text'], add_special_tokens=False)

            encoding_length = len(encode_text)
            encoder_inputs['input_ids'].append(encode_text)
            for key in piece:
                if key not in ['text']:
                    encoder_inputs[key].append([piece[key]]*encoding_length)

        encoder_inputs = self.truncate(encoder_inputs=encoder_inputs)
        # delete shortenable ids
        encoder_inputs.pop("shortenable_ids")
        encoder_inputs = self.concate_parts(input_dict=encoder_inputs)
        encoder_inputs = self.add_special_tokens(encoder_inputs=encoder_inputs)
        # create special input ids
        encoder_inputs['attention_mask'] = [1] *len(encoder_inputs['input_ids'])
        if self.create_token_type_ids:
            encoder_inputs['token_type_ids'] = [0] *len(encoder_inputs['input_ids'])
        # padding
        encoder_inputs = self.padding(input_dict=encoder_inputs, max_len=self.max_seq_length, pad_id_for_inputs=self.tokenizer.pad_token_id)

        if len(encoded_tgt_text) > 0:
            encoder_inputs = {**encoder_inputs, "encoded_tgt_text": encoded_tgt_text}# convert defaultdict to dict
        else:
            encoder_inputs = {**encoder_inputs}
        return encoder_inputs

In [12]:
def load_plm(model_name, model_path):   
    model_config = XLMRobertaConfig.from_pretrained(model_path)
    model = XLMRobertaForMaskedLM.from_pretrained(model_path, config=model_config)
    tokenizer = XLMRobertaTokenizer.from_pretrained(model_path)
    wrapper = MLMTokenizerWrapper
    return model, tokenizer, wrapper

_Load Pre-trained Language Model (PLM)_


In [13]:
plm, tokenizer, wrapper_class = load_plm("xlm", "xlm-roberta-base")

_Prompt Template, Verbalizer and PLM_


In [14]:
prompt_verbalizer_sinhala = ManualVerbalizer(
    classes = [ "Positive","Negative","Neutral","Conflict"],
    label_words={"Positive":["හොඳයි", "ධනාත්මක"], "Negative":["නරකයි", "ඍණාත්මක"], "Neutral":["මධ්යස්ථ"], "Conflict":["ගැටුම්කාරී"]},
    tokenizer = tokenizer
)

prompt_verbalizer_english = ManualVerbalizer(
    classes = [ "Positive","Negative","Neutral","Conflict"],
    label_words={"Positive":["good", "positive","great"], "Negative":["bad", "negative","terrible"], "Neutral":["okay", "neutral"], "Conflict":["conflict"]},
    tokenizer = tokenizer
)

def get_prompttemplate_and_model(template):
    prompt_template = ManualTemplate(text = template, tokenizer = tokenizer)
    prompt_model = PromptForClassification(template = prompt_template, plm = plm, verbalizer = prompt_verbalizer_english, freeze_plm=True)
    
    return prompt_template, prompt_model

## Create Test DataLoaders


In [15]:
X_train, y_train = X_train.values.tolist(), y_train.values.tolist()
X_validation, y_validation = X_validation.values.tolist(), y_validation.values.tolist()
X_test, y_test = X_test.values.tolist(), y_test.values.tolist()

In [16]:
test_examples=[]
test_examples_latin=[]
test_examples_sinhala=[]
test_examples_mixed=[]

for i in range(len(X_test)):
    test_examples.append(InputExample(guid = i, text_a = X_test[i][0], label = y_test[i][0]))

    if(X_test[i][1]=="Latin"):
        test_examples_latin.append(InputExample(guid = i, text_a = X_test[i][0], label = y_test[i][0]))
    elif(X_test[i][1]=="Sinhala"):
        test_examples_sinhala.append(InputExample(guid = i, text_a = X_test[i][0], label = y_test[i][0]))    
    elif(X_test[i][1]=="Mixed"):
        test_examples_mixed.append(InputExample(guid = i, text_a = X_test[i][0], label = y_test[i][0]))

In [17]:
def get_test_data_loaders(prompt_template):
    test_data_loader = PromptDataLoader(
        dataset = test_examples,
        tokenizer = tokenizer,
        template = prompt_template,
        tokenizer_wrapper_class=wrapper_class,
        batch_size=batch_size,
        max_seq_length=max_seq_length,
        truncation=True,
        padding="max_length"
    ).dataloader
    test_data_loader_latin = PromptDataLoader(
        dataset = test_examples_latin,
        tokenizer = tokenizer,
        template = prompt_template,
        tokenizer_wrapper_class=wrapper_class,
        batch_size=batch_size,
        max_seq_length=max_seq_length,
        truncation=True,
        padding="max_length"
    ).dataloader
    test_data_loader_sinhala = PromptDataLoader(
        dataset = test_examples_sinhala,
        tokenizer = tokenizer,
        template = prompt_template,
        tokenizer_wrapper_class=wrapper_class,
        batch_size=batch_size,
        max_seq_length=max_seq_length,
        truncation=True,
        padding="max_length"
    ).dataloader
    test_data_loader_mixed = PromptDataLoader(
        dataset = test_examples_mixed,
        tokenizer = tokenizer,
        template = prompt_template,
        tokenizer_wrapper_class=wrapper_class,
        batch_size=batch_size,
        max_seq_length=max_seq_length,
        truncation=True,
        padding="max_length",
    ).dataloader
    
    return test_data_loader, test_data_loader_latin, test_data_loader_sinhala, test_data_loader_mixed

## Define Evaluation Methods


In [18]:
def predict(promptModel, dataloader):
    if use_cuda: promptModel=promptModel.cuda()
    promptModel.eval()
    allpreds = []
    alllabels = []

    for step, inputs in enumerate(dataloader):
        if use_cuda: inputs = inputs.cuda()
        logits = promptModel(inputs)
        labels = inputs['label']
        alllabels.extend(labels.cpu().tolist())
        allpreds.extend(torch.argmax(logits, dim=-1).cpu().tolist())
    return allpreds,alllabels

In [19]:
def compute_metrics(allpreds,alllabels):
    metric1 = load_metric("precision")
    metric2 = load_metric("recall")
    metric3 = load_metric("f1")
    metric4 = load_metric("accuracy")
    
    predictions, labels = allpreds,alllabels
    precision = metric1.compute(predictions=predictions, references=labels, average="weighted")["precision"]
    recall = metric2.compute(predictions=predictions, references=labels, average="weighted")["recall"]
    f1 = metric3.compute(predictions=predictions, references=labels, average="weighted")["f1"]
    accuracy = metric4.compute(predictions=predictions, references=labels)["accuracy"]
    macro_precision = metric1.compute(predictions=predictions, references=labels, average="macro")["precision"]
    macro_recall = metric2.compute(predictions=predictions, references=labels, average="macro")["recall"]
    macro_f1 = metric3.compute(predictions=predictions, references=labels, average="macro")["f1"]
    return {"accuracy":accuracy, "precision": precision, "recall": recall, "f1": f1, "macro_precision": macro_precision, "macro_recall": macro_recall, "macro_f1": macro_f1}

In [20]:
def evaluate(promptModel, dataloader):
    allpreds,alllabels=predict(promptModel, dataloader)
    return compute_metrics(allpreds,alllabels)

## Test


In [21]:
def test(template, prompt_model, test_data_loader_latin, test_data_loader_sinhala, test_data_loader_mixed, test_data_loader_all):
    t = PrettyTable()
    t.field_names = ['Script', 'Accuracy', 'Precision', 'Recall', 'F1', 'Macro Pr.', 'Macro Re.', 'Macro F1']

    res = evaluate(prompt_model, test_data_loader_latin)
    res = {key : round(res[key], 4) for key in res}
    t.add_row(["Latin", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])
    
    res = evaluate(prompt_model, test_data_loader_sinhala)
    res = {key : round(res[key], 4) for key in res}
    t.add_row(["Sinhala", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])
    
    res = evaluate(prompt_model, test_data_loader_mixed)
    res = {key : round(res[key], 4) for key in res}
    t.add_row(["Mixed", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])
    
    res = evaluate(prompt_model, test_data_loader)
    res = {key : round(res[key], 4) for key in res}
    t.add_row(["Overall", res['accuracy'], res['precision'], res['recall'], res['f1'], res['macro_precision'], res['macro_recall'], res['macro_f1']])
    
    print(template)
    print(t)

In [22]:
sinhala_templates = [
    '{"placeholder": "text_a"}. හැඟීම: {"mask"}.',
    '{"placeholder": "text_a"}. එය {"mask"} විය.',
    '{"placeholder": "text_a"}. මෙය {"mask"} ප්‍රකාශයකි.',
    '{"placeholder": "text_a"}. සාරාංශයක් ලෙස, එය {"mask"} විය.',
    '{"placeholder": "text_a"}. සමස්තයක් වශයෙන්, එය {"mask"} විය.',
    '{"placeholder": "text_a"}. ප්‍රශ්නය: මෙම ප්‍රකාශයේ හැඟීම කුමක්ද? පිළිතුර: {"mask"}.'
]

english_templates = [
    '{"placeholder": "text_a"}. {"mask"}.',
    '{"placeholder": "text_a"}. It was {"mask"}.',
    '{"placeholder": "text_a"}. Sentiment: {"mask"}.',
    '{"placeholder": "text_a"}. In summary, it was {"mask"}.',
    '{"placeholder": "text_a"}. All in all, it was {"mask"}.',
    '{"placeholder": "text_a"}. Question: What is the sentiment of this statement? Answer: {"mask"}.',
    '{"placeholder": "text_a"}. The sentiment or the feeling of the given sentence can be classified as positive, negative or neutral. The classified sentiment of the sentence is {"mask"}.'
]

In [23]:
for template in english_templates:
    prompt_template, prompt_model = get_prompttemplate_and_model(template)
    test_data_loader, test_data_loader_latin, test_data_loader_sinhala, test_data_loader_mixed = get_test_data_loaders(prompt_template)
    test(template, prompt_model, test_data_loader_latin, test_data_loader_sinhala, test_data_loader_mixed, test_data_loader)

tokenizing: 1352it [00:00, 1538.09it/s]
tokenizing: 972it [00:00, 1039.17it/s]
tokenizing: 184it [00:00, 1527.24it/s]
tokenizing: 195it [00:00, 1435.61it/s]


{"placeholder": "text_a"}. {"mask"}.
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Script | Accuracy | Precision | Recall |   F1   | Macro Pr. | Macro Re. | Macro F1 |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Latin  |  0.2469  |   0.2605  | 0.2469 | 0.1135 |   0.1375  |   0.2367  |  0.1026  |
| Sinhala |  0.2717  |    0.68   | 0.2717 | 0.1233 |   0.3169  |   0.2522  |   0.11   |
|  Mixed  |  0.3385  |   0.1146  | 0.3385 | 0.1712 |   0.0846  |    0.25   |  0.1264  |
| Overall |  0.2633  |   0.2826  | 0.2633 | 0.1223 |   0.1488  |   0.2413  |  0.1074  |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+


tokenizing: 1352it [00:00, 1465.72it/s]
tokenizing: 972it [00:00, 1321.35it/s]
tokenizing: 184it [00:00, 1002.82it/s]
tokenizing: 195it [00:00, 974.06it/s]


{"placeholder": "text_a"}. It was {"mask"}.
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Script | Accuracy | Precision | Recall |   F1   | Macro Pr. | Macro Re. | Macro F1 |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Latin  |  0.2562  |   0.3888  | 0.2562 | 0.1237 |   0.1877  |   0.2433  |  0.1078  |
| Sinhala |  0.2663  |   0.0709  | 0.2663 | 0.112  |   0.0666  |    0.25   |  0.1052  |
|  Mixed  |  0.3231  |   0.1111  | 0.3231 | 0.1653 |   0.082   |   0.2386  |  0.1221  |
| Overall |  0.267   |   0.3509  | 0.267  | 0.1273 |   0.176   |   0.2436  |  0.1098  |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+


tokenizing: 1352it [00:00, 1439.58it/s]
tokenizing: 972it [00:00, 1447.96it/s]
tokenizing: 184it [00:00, 1398.69it/s]
tokenizing: 195it [00:00, 1323.28it/s]


...

{"placeholder": "text_a"}. All in all, it was {"mask"}.
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Script | Accuracy | Precision | Recall |   F1   | Macro Pr. | Macro Re. | Macro F1 |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Latin  |  0.2541  |   0.3597  | 0.2541 | 0.1127 |   0.1767  |   0.245   |  0.104   |
| Sinhala |  0.2663  |   0.0709  | 0.2663 | 0.112  |   0.0666  |    0.25   |  0.1052  |
|  Mixed  |  0.3333  |   0.1134  | 0.3333 | 0.1692 |   0.0838  |   0.2462  |  0.125   |
| Overall |  0.267   |   0.3366  | 0.267  | 0.1199 |   0.1706  |   0.246   |  0.1074  |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+


tokenizing: 1352it [00:01, 1290.75it/s]
tokenizing: 972it [00:01, 934.39it/s]
tokenizing: 184it [00:00, 1306.93it/s]
tokenizing: 195it [00:00, 1238.65it/s]


{"placeholder": "text_a"}. Question: What is the sentiment of this statement? Answer: {"mask"}.
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Script | Accuracy | Precision | Recall |   F1   | Macro Pr. | Macro Re. | Macro F1 |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Latin  |  0.2562  |   0.0656  | 0.2562 | 0.1045 |   0.064   |    0.25   |  0.102   |
| Sinhala |  0.2663  |   0.0709  | 0.2663 | 0.112  |   0.0666  |    0.25   |  0.1052  |
|  Mixed  |  0.3385  |   0.1146  | 0.3385 | 0.1712 |   0.0846  |    0.25   |  0.1264  |
| Overall |  0.2692  |   0.0725  | 0.2692 | 0.1142 |   0.0673  |    0.25   |  0.1061  |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+


tokenizing: 1352it [00:01, 961.39it/s]
tokenizing: 972it [00:00, 1040.56it/s]
tokenizing: 184it [00:00, 1052.88it/s]
tokenizing: 195it [00:00, 985.60it/s]


{"placeholder": "text_a"}. The sentiment or the feeling of the given sentence can be classified as positive, negative or neutral. The classified sentiment of the sentence is {"mask"}.
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Script | Accuracy | Precision | Recall |   F1   | Macro Pr. | Macro Re. | Macro F1 |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
|  Latin  |  0.2551  |   0.0654  | 0.2551 | 0.1041 |   0.0639  |   0.249   |  0.1016  |
| Sinhala |  0.2663  |   0.0709  | 0.2663 | 0.112  |   0.0666  |    0.25   |  0.1052  |
|  Mixed  |  0.3385  |   0.1146  | 0.3385 | 0.1712 |   0.0846  |    0.25   |  0.1264  |
| Overall |  0.2685  |   0.0723  | 0.2685 | 0.114  |   0.0672  |   0.2493  |  0.1058  |
+---------+----------+-----------+--------+--------+-----------+-----------+----------+
